<a href="https://colab.research.google.com/github/sourabhnayak04444/REST-API-with-Flask/blob/main/Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask pyngrok -q
!fuser -k 5000/tcp

from flask import Flask, request, jsonify
from threading import Thread
from pyngrok import ngrok
import time

NGROK_AUTH_TOKEN = "34VsGOo7Ji2LEAv2WtAVEBmmiO7_J5HVipuFbFmFTy6ugZHs"
if NGROK_AUTH_TOKEN == "34VsGOo7Ji2LEAv2WtAVEBmmiO7_J5HVipuFbFmFTy6ugZHs":
    print("ERROR: Please replace 'YOUR_RAW_NGROK_TOKEN_HERE' with your actual ngrok Auth Token.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)
PORT = 5000
users = {
    1: {"name": "Alice", "email": "alice@example.com"},
    2: {"name": "Bob", "email": "bob@example.com"},
}
next_id = 3

@app.route('/')
def home():
    return "User Management API is running!"

@app.route('/users', methods=['GET', 'POST'])
def manage_users():
    global next_id
    if request.method == 'POST':
        data = request.json
        if not data or 'name' not in data or 'email' not in data:
            return jsonify({"error": "Missing name or email"}), 400

        user_id = next_id
        new_user = {"name": data['name'], "email": data['email']}
        users[user_id] = new_user
        next_id += 1
        return jsonify({"id": user_id, **new_user}), 201

    user_list = [{"id": uid, **data} for uid, data in users.items()]
    return jsonify(user_list), 200

@app.route('/users/<int:user_id>', methods=['GET', 'PUT', 'DELETE'])
def manage_specific_user(user_id):
    if user_id not in users:
        return jsonify({"error": "User not found"}), 404

    if request.method == 'GET':
        return jsonify({"id": user_id, **users[user_id]}), 200

    elif request.method == 'PUT':
        data = request.json
        if not data:
            return jsonify({"error": "Request body is empty"}), 400

        if 'name' in data:
            users[user_id]['name'] = data['name']
        if 'email' in data:
            users[user_id]['email'] = data['email']

        return jsonify({"message": "User updated", "user": {"id": user_id, **users[user_id]}}), 200

    elif request.method == 'DELETE':
        del users[user_id]
        return jsonify({"message": "User deleted"}), 204

def run_flask():
    app.run(port=PORT, use_reloader=False)

flask_thread = Thread(target=run_flask)
flask_thread.start()
time.sleep(2)
try:
    public_url = ngrok.connect(PORT)
    print(f"Your API is LIVE at: {public_url}")
    print("You can use this URL with Postman or cURL to test your endpoints.")
except Exception as e:
    print(f"ngrok connection failed: {e}")
    print("Please check your NGROK_AUTH_TOKEN.")

In [ ]:
NGROK_URL = "https://archangelical-claudette-rabidly.ngrok-free.dev"
print("--- 1. Check API Status (GET /) ---")
!curl {NGROK_URL}/
print("\n\n--- 2. GET (All Users) ---")
!curl {NGROK_URL}/users

print("\n\n--- 3. PUT (Update User 3) ---")
!curl -X PUT -H "Content-Type: application/json" \
  -d '{"name":"Updated User Name", "email":"updated@email.com"}' \
  {NGROK_URL}/users/3

print("\n\n--- 4. GET (Specific User 3) ---")
!curl {NGROK_URL}/users/3

print("\n\n--- 5. DELETE (Delete User 3) ---")
!curl -X DELETE {NGROK_URL}/users/3

print("\n\n--- 6. GET (All Users - Confirming Delete) ---")
!curl {NGROK_URL}/users